<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=bb8fff0fa99289e710b6d02ab469215a786edfebbe9721425f17c7c57bd22728
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-06 11:46:28
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.43 C
-------------------
Today PnL: -1.39 L (-0.96%)
Current PnL: -21.40 L (-14.02%)
CY Booked + Current PnL: -8.75 L (-5.74%)
-------------------
Total profit:  1.96 L
Total loss:  -23.36 L
-------------------
Total Booked + Current PnL: 18.23 L (14.51%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.84%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 88.39 L (61.74%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.66%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-6.91,67.0,X-LC,5.81,233095.0,22729.0,6060.0,1.43,10.80,2.60,13.68,37.0,3.75,1.64,29.11,XY25,NTT,REFINERIES
78,TTKPRESTIG,770.00,102.13,52.0,M-SC,1.75,87128.0,-13649.0,13740.0,-0.55,-13.54,15.77,0.09,245.0,-0.99,0.61,14.47,OX40N,NTT,DURABLES
79,UNIONBANK,163.00,-5.43,64.0,M-LC,10.63,170595.0,29755.0,14739.0,-0.99,21.13,8.64,31.59,66.0,2.02,1.20,53.21,XY24,NTT,BANKS
2,ABBOTINDIA,35195.00,-9.64,47.0,X-MC,2.56,89235.0,-1443.0,16348.0,1.31,-1.59,18.32,16.44,101.0,-0.09,0.63,18.26,X40,ATH,PHARMA
33,ICICIGI,2252.93,-13.84,57.0,X-MC,5.54,148222.0,12089.0,18498.0,-1.81,8.88,12.48,22.47,91.0,0.65,1.04,23.01,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANPAINT,3465.66,-9.27,69.0,X-LC,4.63,229126.0,-22642.0,75864.0,4.71,-8.99,33.11,21.13,27.0,-0.30,1.61,23.25,X40,ATH,PAINTS
24,EASEMYTRIP,26.40,-16.12,57.0,M-SC,27.27,80894.0,-95921.0,176721.0,4.54,-54.25,218.46,45.70,197.0,-0.54,0.57,5.61,XY24,NTT,TRAVEL
13,BERGEPAINT,680.00,-17.54,57.0,X-MC,1.29,221494.0,-5845.0,53912.0,1.76,-2.57,24.34,21.14,106.0,-0.11,1.56,24.85,XY24,NTT,PAINTS
67,SIS,528.00,2022.28,46.0,H-SC,2.73,85654.0,-25378.0,48986.0,1.48,-22.86,57.19,21.26,156.0,-0.52,0.60,15.73,OX40N,NTT,MISC
58,RELIANCE,1533.00,-6.91,67.0,X-LC,5.81,233095.0,22729.0,6060.0,1.43,10.80,2.60,13.68,37.0,3.75,1.64,29.11,XY25,NTT,REFINERIES


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SFL,1287.0,31.28,45.0,M-SC,4.39,173402.0,-88816.0,158645.0,-6.29,-33.87,91.49,26.63,229.0,-0.56,1.22,7.81,XY24,NTT,MISC
8,ATULAUTO,844.0,3381.08,35.0,M-SC,3.30,156315.0,-40817.0,130648.0,-3.79,-20.71,83.58,45.57,236.0,-0.31,1.10,10.82,XY24,NTT,AUTO
19,COFFEEDAY,80.0,-49.70,35.0,L-SC,27.74,74784.0,-38765.0,78815.0,-3.76,-34.14,105.39,35.27,268.0,-0.49,0.53,83.04,XR,NTT,HOTELS
59,REPCOHOME,880.0,-53.06,65.0,H-SC,7.26,268305.0,-18200.0,275522.0,-3.56,-6.35,102.69,89.82,134.0,-0.07,1.89,41.95,XY24,NTT,FINANCE
71,SYMPHONY,1306.0,-44.37,46.0,M-SC,17.03,118725.0,-52416.0,52358.0,-3.35,-30.63,44.10,-0.03,196.0,-1.00,0.84,3.51,OX40N,NTT,DURABLES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,24.68,43.0,M-LC,10.42,203058.0,-2018.0,114586.0,-1.38,-0.98,56.43,54.89,52.0,-0.02,1.43,22.87,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,102.13,52.0,M-SC,1.75,87128.0,-13649.0,13740.0,-0.55,-13.54,15.77,0.09,245.0,-0.99,0.61,14.47,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,46.0,H-SC,2.70,222579.0,-47088.0,83423.0,-0.49,-17.46,37.48,13.47,138.0,-0.56,1.57,13.17,XY24,NTT,PAINTS
17,CERA,9475.0,-22.43,50.0,H-SC,1.80,141358.0,-34545.0,76574.0,-2.36,-19.64,54.17,23.89,149.0,-0.45,0.99,22.08,OX40N,NTT,CERAMICS
67,SIS,528.0,2022.28,46.0,H-SC,2.73,85654.0,-25378.0,48986.0,1.48,-22.86,57.19,21.26,156.0,-0.52,0.60,15.73,OX40N,NTT,MISC
42,JCHAC,2282.0,19900.00,61.0,M-SC,1.51,97731.0,-30074.0,30062.0,-0.13,-23.53,30.76,-0.01,232.0,-1.00,0.69,10.67,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-5.43,64.0,M-LC,10.63,170595.0,29755.0,14739.0,-0.99,21.13,8.64,31.59,66.0,2.02,1.2,53.21,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,49.16,56.0,M-MC,9.90,234864.0,9902.0,157758.0,-1.30,4.40,67.17,74.53,192.0,0.06,1.65,38.07,XY24,BTT,STEEL
46,KPIGREEN,731.05,9.02,59.0,H-SC,7.41,129931.0,4634.0,54298.0,-3.09,3.70,41.79,47.03,141.0,0.09,0.91,62.69,MH,ATH,POWER
79,UNIONBANK,163.00,-5.43,64.0,M-LC,10.63,170595.0,29755.0,14739.0,-0.99,21.13,8.64,31.59,66.0,2.02,1.20,53.21,XY24,NTT,BANKS
85,WHIRLPOOL,2270.00,-39.40,57.0,M-SC,5.41,125598.0,10800.0,83246.0,-0.67,9.41,66.28,81.92,223.0,0.13,0.88,48.69,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.52,53.0,H-SC,5.84,129702.0,6366.0,125266.0,-0.85,5.16,96.58,106.72,119.0,0.05,0.91,29.75,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-39.40,57.0,M-SC,5.41,125598.0,10800.0,83246.0,-0.67,9.41,66.28,81.92,223.0,0.13,0.88,48.69,XR,NTT,DURABLES
46,KPIGREEN,731.05,9.02,59.0,H-SC,7.41,129931.0,4634.0,54298.0,-3.09,3.70,41.79,47.03,141.0,0.09,0.91,62.69,MH,ATH,POWER
36,INDIAMART,4810.62,-53.52,53.0,H-SC,5.84,129702.0,6366.0,125266.0,-0.85,5.16,96.58,106.72,119.0,0.05,0.91,29.75,AR,ATH,MISC
31,HINDZINC,730.22,24.68,43.0,M-LC,10.42,203058.0,-2018.0,114586.0,-1.38,-0.98,56.43,54.89,52.0,-0.02,1.43,22.87,X5K,ATH,METALS
86,WIPRO,420.00,-15.61,41.0,M-LC,5.50,148068.0,-2877.0,112324.0,0.38,-1.91,75.86,72.51,53.0,-0.03,1.04,4.00,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,21.0,X-LC,28.48,147560.0,-128963.0,239032.0,-2.52,-46.64,161.99,39.81,54.0,-0.54,1.04,4.00,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-41.58,25.0,X-SC,6.95,87274.0,-41396.0,86689.0,-1.37,-32.17,99.33,35.20,219.0,-0.48,0.61,0.17,X40,NTT,FOOTWEAR
44,JSWINFRA,342.00,-23.55,28.0,X-MC,4.69,191700.0,-8647.0,39145.0,-1.97,-4.32,20.42,15.23,178.0,-0.22,1.35,26.54,X40N,NTT,REALTY
23,DMART,5391.45,-19.11,31.0,X-LC,8.30,206080.0,-4327.0,63493.0,-1.45,-2.06,30.81,28.12,38.0,-0.07,1.45,21.08,X40N,ATH,FMCG
53,QUESS,986.00,-50.96,32.0,X-SC,37.76,48929.0,-16077.0,167004.0,-2.80,-24.73,341.32,232.18,198.0,-0.10,0.34,0.00,XY24,NTT,MISC


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17907.65,-31.08,36.0,X-MC,0.92,199635.0,-1185.0,68974.0,-0.69,-0.59,34.55,33.76,80.0,-0.02,1.40,4.58,X40,ATH,FMCG
41,ITC,452.00,-38.34,45.0,X-LC,1.13,199145.0,-993.0,21886.0,-0.40,-0.50,10.99,10.44,4.0,-0.05,1.40,5.04,X40,NTT,FMCG
66,SIEMENS,4671.50,-3.47,45.0,H-LC,1.25,155050.0,-31045.0,78517.0,-0.60,-16.68,50.64,25.51,15.0,-0.40,1.09,14.26,AR,ATH,ELECTRICAL
28,HAVELLS,2069.16,-8.02,38.0,X-MC,3.04,306747.0,-21244.0,127791.0,-1.73,-6.48,41.66,32.48,92.0,-0.17,2.16,4.21,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-6.66,64.0,X-LC,4.12,244790.0,10646.0,37967.0,-1.91,4.55,15.51,20.77,86.0,0.28,1.72,11.78,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-50.96,32.0,X-SC,37.76,48929.0,-16077.0,167004.0,-2.80,-24.73,341.32,232.18,198.0,-0.10,0.34,0.00,XY24,NTT,MISC
12,BATAINDIA,2096.00,-41.58,25.0,X-SC,6.95,87274.0,-41396.0,86689.0,-1.37,-32.17,99.33,35.20,219.0,-0.48,0.61,0.17,X40,NTT,FOOTWEAR
51,PAGEIND,51769.93,-29.36,34.0,X-MC,7.23,79820.0,-3690.0,23723.0,-0.56,-4.42,29.72,23.98,82.0,-0.16,0.56,1.14,X40,ATH,APPARELS
20,COLPAL,3726.84,-6.96,40.0,X-MC,7.35,217190.0,-46175.0,155486.0,-0.86,-17.53,71.59,41.51,84.0,-0.30,1.53,1.92,XY25,ATH,FMCG
3,ACC,3906.00,-38.50,44.0,X-MC,2.44,183300.0,-54651.0,207294.0,0.02,-22.97,113.09,64.15,174.0,-0.26,1.29,2.82,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-50.96,32.0,X-SC,37.76,48929.0,-16077.0,167004.0,-2.80,-24.73,341.32,232.18,198.0,-0.10,0.34,0.00,XY24,NTT,MISC
51,PAGEIND,51769.93,-29.36,34.0,X-MC,7.23,79820.0,-3690.0,23723.0,-0.56,-4.42,29.72,23.98,82.0,-0.16,0.56,1.14,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.46,41.0,X-SC,4.32,81103.0,-64057.0,139984.0,-0.42,-44.13,172.60,52.31,136.0,-0.46,0.57,8.56,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-41.58,25.0,X-SC,6.95,87274.0,-41396.0,86689.0,-1.37,-32.17,99.33,35.20,219.0,-0.48,0.61,0.17,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-9.64,47.0,X-MC,2.56,89235.0,-1443.0,16348.0,1.31,-1.59,18.32,16.44,101.0,-0.09,0.63,18.26,X40,ATH,PHARMA


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-29.01,48.0,X-LC,9.86,282489.0,-63467.0,130171.0,0.50,-18.35,46.08,19.28,1.0,-0.49,1.99,4.43,X40,ATH,IT
39,INFY,2275.00,-19.60,50.0,X-LC,5.12,313834.0,946.0,170726.0,0.37,0.30,54.40,54.87,3.0,0.01,2.21,8.61,X40,BTT,IT
41,ITC,452.00,-38.34,45.0,X-LC,1.13,199145.0,-993.0,21886.0,-0.40,-0.50,10.99,10.44,4.0,-0.05,1.40,5.04,X40,NTT,FMCG
83,VBL,671.64,-16.58,54.0,X-LC,7.20,300498.0,-15344.0,128012.0,-0.79,-4.86,42.60,35.67,5.0,-0.12,2.11,8.32,X40N,ATH,FMCG
1,ABB,7934.00,-39.57,54.0,H-LC,5.08,252672.0,-8947.0,128155.0,0.38,-3.42,50.72,45.57,7.0,-0.07,1.78,7.51,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-49.70,35.0,L-SC,27.74,74784.0,-38765.0,78815.0,-3.76,-34.14,105.39,35.27,268.0,-0.49,0.53,83.04,XR,NTT,HOTELS
7,ASIANTILES,137.00,7438.89,67.0,L-SC,10.63,82726.0,-11084.0,87706.0,-0.97,-11.82,106.02,81.67,269.0,-0.13,0.58,60.20,XR,NTT,CERAMICS
49,MASFIN,398.61,-19.15,49.0,H-SC,8.36,91860.0,-6120.0,27723.0,-2.11,-6.25,30.18,22.05,152.0,-0.22,0.65,33.53,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-39.40,57.0,M-SC,5.41,125598.0,10800.0,83246.0,-0.67,9.41,66.28,81.92,223.0,0.13,0.88,48.69,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.52,53.0,H-SC,5.84,129702.0,6366.0,125266.0,-0.85,5.16,96.58,106.72,119.0,0.05,0.91,29.75,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VAIBHAVGBL,521.00,74.10,65.0,H-SC,2.31,151152.0,-31623.0,142687.0,-2.39,-17.30,94.40,60.77,125.0,-0.22,1.06,35.38,XR,NTT,JEWELLERY
59,REPCOHOME,880.00,-53.06,65.0,H-SC,7.26,268305.0,-18200.0,275522.0,-3.56,-6.35,102.69,89.82,134.0,-0.07,1.89,41.95,XY24,NTT,FINANCE
62,SAMMAANCAP,326.00,-197.73,71.0,M-SC,33.12,168642.0,19422.0,124761.0,0.62,13.02,73.98,96.62,208.0,0.16,1.19,82.51,XY25,NTT,FINANCE
68,SONACOMS,806.63,-31.93,67.0,M-SC,6.91,84709.0,-16450.0,56450.0,0.05,-16.26,66.64,39.54,202.0,-0.29,0.60,19.45,AR,ATH,AUTO
46,KPIGREEN,731.05,9.02,59.0,H-SC,7.41,129931.0,4634.0,54298.0,-3.09,3.70,41.79,47.03,141.0,0.09,0.91,62.69,MH,ATH,POWER


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.73
1,25,44.43
2,50,75.52


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.71
LC    33.19
MC    23.08
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.97
X40      16.25
X40N     12.40
XY25     11.97
XR        9.82
AR        7.86
OX40N     6.53
X200      1.79
X5K       1.43
SR        1.05
MH        0.91
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.19
X-LC    22.48
X-MC    18.77
M-SC    12.62
X-SC     5.46
M-LC     5.08
H-LC     4.66
H-MC     2.35
M-MC     1.65
L-SC     1.44
L-LC     0.97
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.33,-3.80,37.36
IT,12.43,-19.05,82.82
FINANCE,12.18,-9.34,58.90
MISC,7.67,-19.04,79.09
ELECTRICAL,5.79,-9.87,48.82
PAINTS,5.61,-15.95,33.24
BANKS,4.01,-18.63,87.64
INSURANCE,3.84,-1.28,39.04
AUTO,2.74,-47.93,109.66


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3316094.0,22
XR,1258912.0,13
AR,1171961.0,9
X40,845272.0,12
X40N,705141.0,9
OX40N,557109.0,9
XY25,472826.0,8
SR,272245.0,2
X5K,114586.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3392661.0,24
M-SC,1254409.0,15
X-MC,1207242.0,14
X-LC,1082526.0,13
X-SC,571014.0,6
M-LC,286393.0,4
H-LC,276770.0,3
L-SC,256616.0,3
H-MC,254041.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1190418.0      6
           AR         841270.0      5
           XR         789623.0      7
M-SC       XY24       725692.0      6
X-MC       XY24       566384.0      4
X-LC       X40        460989.0      5
X-SC       X40N       317321.0      4
X-LC       XY24       314631.0      2
X-MC       X40        297594.0      6
H-SC       SR         272245.0      2
           OX40N      244807.0      3
X-LC       X40N       229472.0      3
H-LC       AR         206672.0      2
X-MC       XY25       184916.0      2
H-MC       XY24       179468.0      1
X-SC       XY24       167004.0      1
L-SC       XR         166521.0      2
X-MC       X40N       158348.0      2
M-MC       XY24       157758.0      1
M-SC       OX40N      147634.0      4
           XR         132303.0      2
           XY25       124761.0      1
           AR         124019.0      2
M-LC       X5K        114586.0      1
           XR         112324.0      1
L-SC       OX40N       90095.0      1
X-SC       X40         86689.0      1
X-LC       XY25        77434.0      3
H-MC       OX40N       74573.0      1
H-LC       X200        70098.0      1
L-MC       XR          58141.0      1
H-SC       MH          54298.0      1
M-LC       XY25        44744.0      1
L-LC       XY25        40971.0      1
M-LC       XY24        14739.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
